In [ ]:
import h5py, sys, os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm

In [ ]:
sys.path.append('/mnt/cbis/home/benedikt/.local/dragonfly/utils/py_src')

In [ ]:
import writeemc
import detector
import reademc

In [ ]:
path_to_data = "../../data/sparse/" # scratch/sparse on Maxwell
path_to_aux = "../../data/aux/" # scratch/benedikt/aux on Maxwell
path_to_geometry = "../../geometry/"
path_to_recons = "../../data/recons/"

## Reading metadata

In [ ]:
with h5py.File(path_to_aux + "sparse_hits_metadata.h5", "r") as f:
    cells = f['cells'][:]
    score = f['scores'][:]
    train = f['train'][:]

## Reading 2D EMC ring classes (round 0)

In [ ]:
emc_folder = path_to_recons + "ring_0000/"
emc_output = emc_folder + "data/r0/output_060.h5"
emc_photons = emc_folder + 'photons.txt'
emc_detector = path_to_recons + "det/det_2145_lowq1.h5"

In [ ]:
photons_list = list(np.loadtxt(emc_photons, dtype=str))
photons_list = [emc_folder + p for p in photons_list]

In [ ]:
det = detector.Detector(emc_detector, mask_flag=True)
emc = reademc.EMCReader(photons_list, det)

In [ ]:
with h5py.File(emc_output,'r') as f:
    occupancies = f['occupancies'][:]
    likelihood = f['likelihood'][:]
    orientations = f['orientations'][:]
    images = f['intens'][:]
    scale = f['scale'][:]

In [ ]:
modes = orientations % occupancies.shape[1]
blacklisted = (occupancies == 0).all(axis=1)
ndata = len(modes) - blacklisted.sum()

In [ ]:
images[images<1e-21] = 0

In [ ]:
N = occupancies.shape[1]
ncols = 5
nrows = N//ncols
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*4,nrows*4), dpi=200)
for i in range(N):
    c,r = i//ncols, i%ncols
    axes[c,r].axis('off')
    nclass = ((modes==i) & (~blacklisted)).sum()
    axes[c,r].set_title("%d/%d [%.1f%%]" %(nclass,ndata, nclass/ndata*100.))
    axes[c,r].imshow(images[i], norm=colors.LogNorm(vmin=0.0001, vmax=3), cmap='cividis')
    axes[c,r].text(0,0, "%d" %i, color='r')
plt.show()

## Select fiber-like diffraction patterns

In [ ]:
fiber = (modes == 10)
cellid_fiber  = cells[fiber]
scores_fiber  = score[fiber]
trainid_fiber = train[fiber]
index_fiber   = np.where(fiber)[0]

In [ ]:
images_fiber  = np.array([emc.get_frame(i) for i in index_fiber])
modules_fiber = np.array([emc.get_frame(i, raw=True) for i in index_fiber])

In [ ]:
nrphot_fiber = np.array([images_fiber[i][np.bool8(det.mask)].sum() for i in range(images_fiber.shape[0])])

In [ ]:
first  = np.argsort(nrphot_fiber)[::-1][:50]

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=10, figsize=(10,5), dpi=300)
fig.suptitle("Fiber-like features - strongest (0-49)", y=0.95)
cmap = cm.get_cmap('inferno')
cmap.set_bad('0.75')
cmap.set_under('1.0')
for j in range(50):
    c,r = j//10, j%10
    axes[c,r].set_xticks([])
    axes[c,r].set_yticks([])
    axes[c,r].set_title("nphot=%d" %(nrphot_fiber[first[j]]), color='0.5', fontsize=5, pad=2)
    mimg = np.ma.masked_array(np.float64(images_fiber[first[j]]), mask=1-det.mask)
    mimg.data[mimg.data==0] = 1e-10
    axes[c,r].imshow(mimg, norm=colors.LogNorm(vmin=1, vmax=100), cmap=cmap)
#plt.savefig("../../plots/r%04d_best_shots.png" %run, bbox_inches='tight')
plt.show()

## Radial averages

In [ ]:
import spimage

In [ ]:
distance = 0.705 #[m]
wavelength = 1240. / 6010. #[nm]
pixelsize = 200e-6 #[m]
q_to_x = lambda q: np.tan(2.*np.arcsin(wavelength * q / 2.)) * distance / pixelsize
x_to_q = lambda x: (2*np.sin(np.arctan2(x*pixelsize, distance)/2.)) / wavelength 

In [ ]:
NUM_COLORS = occupancies.shape[1]
cmap = plt.get_cmap('tab20')
fig = plt.figure(figsize=(10,5), dpi=200)
ax = fig.add_subplot(111)
ax.set_prop_cycle(color=[cmap(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])
for i in range(occupancies.shape[1]):
    centers, radial = spimage.radialMeanImage(images[i], cx=images.shape[2]//2, cy=images.shape[1]//2, output_r=True)
    ax.plot(x_to_q(centers), radial, label='%d' %i)
ax.set_ylim(0, 0.1)
ax.axvspan(1./(11), 1./(7), color='0.75', ls='-', alpha=0.5)
ax.axvline(1./(8.4), color='k', ls=':')
ax.legend(frameon=0)
ax.set_xlabel(r'Radial spatial frequency [nm$^{-1}$]')
plt.show()